Разработка простых веб-приложений на различных фреймворках — отличный способ понять особенности каждого из них. Мы рассмотрим три популярных фреймворка для создания веб-приложений на Python: Django, Flask и FastAPI. Для сравнения, разработаем одно и то же приложение на каждом из них, а затем проведем анализ их особенностей.

**Задача**

Разработаем простое веб-приложение, которое выполняет следующие функции:

Отображает главную страницу с формой для ввода текста.
Принимает текст от пользователя и сохраняет его в базе данных.
Отображает список всех сохраненных текстов.
Для начала давайте определим структуру базы данных. Она будет содержать одну таблицу с двумя полями: идентификатор записи и сам текст.

**Приложение** **на** **FastAPI**

FastAPI — это современный асинхронный фреймворк, основанный на стандартах Python типа аннотаций и Pydantic. Он обеспечивает высокую производительность и простоту разработки API.

Документация:https://fastapi-tutorial.readthedocs.io/en/latest/

**Установка**



In [ ]:
pip install fastapi uvicorn sqlalchemy aioredis

**Создание** **приложения**

Создадим файл main.py:

In [ ]:
from fastapi import FastAPI, Request, Form
from fastapi.responses import HTMLResponse
from fastapi.templating import Jinja2Templates
from sqlalchemy import create_engine, Column, Integer, String
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

Base = declarative_base()

class Text(Base):
    __tablename__ = 'texts'
    id = Column(Integer, primary_key=True)
    text = Column(String)

engine = create_engine('sqlite:///db.sqlite')
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)
Base.metadata.create_all(bind=engine)

app = FastAPI()
templates = Jinja2Templates(directory="templates")

@app.get("/", response_class=HTMLResponse)
async def read_root(request: Request):
    db = SessionLocal()
    texts = db.query(Text).order_by(Text.id.desc()).all()
    db.close()
    return templates.TemplateResponse("index.html", {"request": request, "texts": texts})

@app.post("/add-text/", response_class=HTMLResponse)
async def add_text(request: Request, text: str = Form(...)):
    db = SessionLocal()
    new_text = Text(text=text)
    db.add(new_text)
    db.commit()
    db.close()
    return templates.TemplateResponse("index.html", {"request": request}, status_code=200)

**Шаблон**

Создадим шаблон templates/index.html:

In [ ]:
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>FastAPI App</title>
</head>
<body>
    <h1>Главная страница</h1>
    <form action="/add-text/" method="post">
        <textarea name="text"></textarea><br>
        <button type="submit">Сохранить</button>
    </form>
    <hr>
    <h2>Список текстов</h2>
    <ul>
        {% for text in texts %}
            <li>{{ text.text }}</li>
        {% endfor %}
    </ul>
</body>
</html>

Запустим сервер:

In [ ]:
uvicorn main:app --reload

Откроется главная страница по адресу http://127.0.0.1:8000/, где можно вводить тексты и сохранять их.

